In [33]:
import pandas as pd

df = pd.read_csv('data.csv')
df

label                                             review
0        0                          味道一般，酸辣土豆丝酸的没法吃。。。肝尖也做的一般
1        0  订的菜没有了，换了个便宜的菜，说了多退少补，结果没把钱给我退回来。诚信很有问题。锅包肉又硬又...
2        0                                       太慢,太慢,太慢,不好吃
3        0                                 商家距离近，但速度太慢，饼子都凉了。
4        0  久久鸭的口味偏颇，辣味不是很浓。总体而言还行。但是没有餐具，虽然我的单里说明要餐具了，这是一...
..     ...                                                ...
195      0                                      两个小时才送到,我是真苦了
196      1         晚了半小时左右吧……不过最近这么冷还是辛苦了，估计大家都不想出门吧。超级好吃！推荐！
197      0                        没吃过这么难吃的水煮牛肉，刚送来吃一口就扔了。有股膻味
198      0                                        送餐慢，肉少，味道不好
199      0                             下错单了,不过冷面味道不错,但是没有给我小菜

[200 rows x 2 columns]

In [45]:
PROMPT = """
您的任务是分析淘宝店铺中买家评论的情感类型(neg、pos)。

<comment_content>
{{COMMENT_CONTENT}}
</comment_content>

请根据以下准则分析此评论的情感：

"neg"：评论文本中包含强烈的负面情绪、沮丧或不满。

"pos"：评论文本表达了满意、感激或积极情绪


分析后，请以以下 JSON 格式提供最终结果：

<answer>
{
"sentiment": "[neg/pos]",
}
</answer>
"""

In [48]:
%%time

import ollama

COMMENT_CONTENT = "11点14订餐，13点20饭才到，2个小时才把我的午饭送到，而且还是打了2次客服电话，1次投诉电话才给送来，要是不打电话都不知道几点能吃上午饭？"

response = ollama.chat(model='qwen2:7b', messages=[
          {'role': 'system', 'content': PROMPT},
          {'role': 'user', 'content': COMMENT_CONTENT},
        ])

result = response['message']['content']
print(result)

{
"sentiment": "neg"
}
CPU times: user 1.94 ms, sys: 1.33 ms, total: 3.27 ms
Wall time: 329 ms


In [63]:
%%time

import pandas as pd
import ollama
from tqdm import tqdm
PROMPT = """
您的任务是分析淘宝店铺中买家评论的情绪和类型。您的目标是确定评论情感(neg、pos)。

<comment_content>
{{COMMENT_CONTENT}}
</comment_content>

请根据以下准则分析此评论的情感：

"neg"：评论文本中包含强烈的负面情绪、沮丧或不满。

"pos"：评论文本表达了满意、感激或积极情绪


分析后，请以以下 JSON 格式提供最终结果：

<answer>
{
"sentiment": "[neg/pos]",
}
</answer>
"""


#考虑到大模型回答的随机性，很容易报错。
#使用tryexcept增强代码的鲁棒性
def sentiment_analysis(comment_text):
    try:
        response = ollama.chat(model = 'qwen2:7b', 
                               messages = [
                                       {'role': 'system', 'content': PROMPT},
                                       {'role': 'user', 'content': comment_text}
                                   ])
        result = response['message']['content']
        if 'pos' in result:
            senti = 'pos'
        elif 'neg' in result:
            senti = 'neg'
        else:
            senti = 'NA'
        return senti
    except:
        return 'NA'

    

sentis = []
df = pd.read_csv('data.csv')
for review in tqdm(df['review']):
    senti = sentiment_analysis(review)
    sentis.append(senti)
    
df['sentiment'] = sentis
df.to_csv('result.csv', index=False)
df

100%|█████████████████████████████████████████| 200/200 [01:35<00:00,  2.10it/s]

CPU times: user 655 ms, sys: 170 ms, total: 825 ms
Wall time: 1min 35s


label                                             review sentiment
0        0                          味道一般，酸辣土豆丝酸的没法吃。。。肝尖也做的一般       neg
1        0  订的菜没有了，换了个便宜的菜，说了多退少补，结果没把钱给我退回来。诚信很有问题。锅包肉又硬又...       neg
2        0                                       太慢,太慢,太慢,不好吃       neg
3        0                                 商家距离近，但速度太慢，饼子都凉了。       neg
4        0  久久鸭的口味偏颇，辣味不是很浓。总体而言还行。但是没有餐具，虽然我的单里说明要餐具了，这是一...       neg
..     ...                                                ...       ...
195      0                                      两个小时才送到,我是真苦了       neg
196      1         晚了半小时左右吧……不过最近这么冷还是辛苦了，估计大家都不想出门吧。超级好吃！推荐！       pos
197      0                        没吃过这么难吃的水煮牛肉，刚送来吃一口就扔了。有股膻味       neg
198      0                                        送餐慢，肉少，味道不好       neg
199      0                             下错单了,不过冷面味道不错,但是没有给我小菜       neg

[200 rows x 3 columns]

In [65]:
df['sentiment'].value_counts()

sentiment
neg    123
pos     77
Name: count, dtype: int64

In [80]:
expression = "(label == 1) & (sentiment == 'pos') | (label == 0) & (sentiment == 'neg')"
correct_ratio = len(df.query(expression))/ len(df)
print(f'准确率: {correct_ratio*100}%')

准确率: 89.5%


准确率: 89.5%
